# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [1]:
import os
import shutil
import numpy as np
from tqdm.notebook import tqdm

import torch
import torchvision
import torchvision.transforms as transforms

import binarybrain as bb

異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [2]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLutSimple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, net_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 1
mini_batch_size       = 64
frame_modulation_size = 15

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [3]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [4]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([1024]),
            bb.DifferentiableLut([420]),
            bb.DifferentiableLut([70]),
            bb.Reduce(integrate_size=7),
            bb.BinaryToReal(frame_modulation_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam()

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [5]:
#bb.load_networks(data_path, net)

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net, keep_olds=3)


epoch[0] : loss=1.741150 accuracy=0.902600


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [6]:
# export verilog
bb.load_networks(data_path, net)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net)

# Simulation用ファイルに上書きコピー
shutil.copyfile(output_velilog_file, sim_velilog_file)

load : ./data/MnistDifferentiableLutSimple\20210110_174836


'../../verilog/mnist\\MnistLutSimple.v'

In [7]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [8]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

load : ./data/MnistDifferentiableLutSimple\20210110_174836


### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、6つの入力と接続されており、layer0 の出力ノードは 1024 個あるので、1024x6 の行列が取得できます。

In [9]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(1024, 6)


array([[285, 227, 432, 459, 774, 502],
       [246,   4, 148, 633, 287, 638],
       [590, 679, 175, 507, 123, 196],
       ...,
       [652, 689, 385, 238, 444, 693],
       [214, 732, 514,  31,  30, 410],
       [548, 372, 435, 273,   8, 598]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [10]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(1024, 64)


array([[ True,  True,  True, ..., False, False, False],
       [ True, False,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ..., False, False,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True, False,  True]])

### 重み行列を覗いてみる

6入力のLUTモデルなので $ 2^6 = 64 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [11]:
W = layer0.W().numpy()
print(W.shape)
W

(1024, 64)


array([[0.52673465, 0.4871359 , 0.51701593, ..., 0.49071878, 0.46445772,
        0.49906301],
       [0.50419724, 0.4626906 , 0.5000362 , ..., 0.5185171 , 0.52558106,
        0.50891495],
       [0.47036353, 0.4801502 , 0.49258187, ..., 0.54539627, 0.53642666,
        0.54357857],
       ...,
       [0.5136678 , 0.46920788, 0.5341977 , ..., 0.46911353, 0.49426067,
        0.4878877 ],
       [0.4639551 , 0.49639666, 0.43498743, ..., 0.50456274, 0.5203207 ,
        0.52957636],
       [0.523955  , 0.48723742, 0.48753798, ..., 0.52356726, 0.47923124,
        0.5170121 ]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [12]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(1024, 64)


array([[ 9.6698239e-04, -1.3669550e-03,  3.9427148e-04, ...,
        -1.0385497e-05,  1.7828708e-05, -2.9141827e-06],
       [ 3.0213865e-03,  1.0511318e-03,  1.0071289e-03, ...,
        -5.5897381e-05, -8.0060709e-05, -1.8632458e-05],
       [-1.5846919e-05, -5.2819378e-06,  2.1643995e-05, ...,
         2.6980979e-07,  1.1088641e-06,  3.6962206e-07],
       ...,
       [ 2.2795433e-03,  6.6156563e-04,  6.1904988e-04, ...,
        -9.2895361e-06, -2.0213771e-05, -4.5025454e-06],
       [ 2.8453494e-04,  1.2742218e-03,  9.4844814e-05, ...,
         4.0643914e-05, -5.8146943e-06,  1.3547966e-05],
       [ 2.6619015e-04,  2.4333212e-04,  5.9101474e-04, ...,
         4.7959602e-06, -5.2254698e-05, -3.7841070e-05]], dtype=float32)

In [13]:
import pickle

In [14]:
data = pickle.dumps(net[0])

In [15]:
pickle.loads(data)

In [16]:
data = pickle.dumps(net)
x_net = pickle.loads(data)

In [17]:
net = x_net

In [18]:
data = pickle.dumps(net[4])
m = pickle.loads(data)
print(type(m))

<class 'binarybrain.models.Reduce'>


In [19]:
print(type(net[4]))

<class 'binarybrain.models.Reduce'>
